In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid
from scipy.signal import convolve2d
from scipy.integrate import quad
from scipy.fft import fft2, ifft2, fftshift, ifftshift

# Physics-related Packages
import astropy.constants as const
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from astropy.io import fits
from astropy.constants import M_sun

In [ ]:
Target_cat_name = "/users_path/merger_trace/data/tng300/tng300_targetcat/TargetHalo_Cat_092.hdf5"
Cutout_Path = "/users_path/merger_trace/data/tng300/tng300_halos/"
SavePath = "/home/chuiyang/merger_trace/data/tng300/tng300_aperture_mass_maps/"

In [ ]:
# preparations for read box info from the url
import requests

baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"API KEY"}

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers, verify= True)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [4]:
# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG300-1')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

In [5]:
# Sim Box parameters
Snap_Index = 92 # the snapshots index in the total 99 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap
DM_Particle_Mass = sim['mass_dm'] # unit: 10^10 M_{sun}/h, mass for a single DM particle

# Short description of cosmological parameters Planck2015
h_atz = cosmo.H(Redshift).value/100 # unit 100km/[s*Mpc]
a_atz = 1/(1+Redshift)

In [ ]:
# Unit Conversion
# 1 solar mass (M_sun) in kilograms
mass_sun_in_kg = M_sun.value  # Value in kg

# Convert meters to kiloparsecs (kpc)
meters_to_kpc = ((1 * u.m).to(u.kpc)).value

kpc_to_ckpcperh = h_atz/a_atz

M_to_e10Mperh = h_atz*1e-10

In [7]:
def get_subhalo_maxM(Halo_ID, Sub_GrNr, Sub_Mass):
    
    """
    Finds the maximum subhalo mass for a given halo.
    
    Parameters:
    - Halo_ID: Array of halo IDs.
    - Sub_GrNr: Array indicating the group number each subhalo belongs to.
    - Sub_MassType: Array of subhalo masses.

    Returns:
    - Tuple (Subhalo_MaxMass, Subhalo_Count): 
      - Subhalo_MaxMass: Maximum subhalo mass for each halo.
      - Subhalo_Count: Number of subhalos for each halo.
    """
        
    find_Sub = np.where(Sub_GrNr == Halo_ID)[0]
    find_Sub_Mass = Sub_Mass[find_Sub]
    CenterSub_Index = np.where(Sub_GrNr == Halo_ID)[0][np.argmax(find_Sub_Mass)]
    
    return np.max(find_Sub_Mass), find_Sub.shape[0], CenterSub_Index

def Get_HaloIDs(TargetHalo_cat, SubhaloMassDef):
    """
    Extracts halo IDs and their properties from the HDF5 catalog.

    Parameters:
    - TargetHalo_cat: Path to the HDF5 file containing the target halo catalog.

    Returns:
    - Target_Halo_IDs: List of selected halo IDs.
    - Subhalo_MaxMasses: Maximum subhalo masses for each selected halo.
    - Target_Halo_Rs_Crit200: Halo critical radius R_Crit200.
    - Target_GroupPoses: Positions of the selected halos.
    - Galaxy_nums: Number of subhalos per halo.
    """

    with h5py.File(TargetHalo_cat, 'r') as Target_hdf:
        FOF_Halo_IDs =  Target_hdf['Group/FOF_Halo_IDs'][:]
        GroupFirstSub = Target_hdf['Group/GroupFirstSub'][:]
        GroupPos = Target_hdf['Group/GroupPos'][:]
        Group_R_Crit200 = Target_hdf['Group/Group_R_Crit200'][:]
        SubhaloGrNr = Target_hdf['Subhalo/SubhaloGrNr'][:]
        # SubhaloMassType =  Target_hdf['Subhalo/SubhaloMassType'][:]
        SubhaloMass =  Target_hdf[f'Subhalo/{SubhaloMassDef}'][:]
        SubhaloSpin = Target_hdf['Subhalo/SubhaloSpin'][:]
        SubhaloVelDisp = Target_hdf['Subhalo/SubhaloVelDisp'][:]
        SubhaloSFR = Target_hdf['Subhalo/SubhaloSFR'][:]
        Subhalo_IDs = Target_hdf['Subhalo/Subhalo_IDs'][:]

    # Get Halo IDs and Halo Radius
    Indices_HaloWithSub = np.where( GroupFirstSub != -1)[0]
    Target_Halo_IDs = FOF_Halo_IDs[Indices_HaloWithSub]
    Target_Halo_Rs_Crit200 = Group_R_Crit200[Indices_HaloWithSub]
    Target_GroupPoses = GroupPos[Indices_HaloWithSub]

    # Get Halo Max Subhalo Mass
    Subhalo_MaxMasses = np.zeros(Target_Halo_IDs.shape)
    Galaxy_nums = np.zeros(Target_Halo_IDs.shape)
    Center_SubhaloVelDisp = np.zeros(Target_Halo_IDs.shape)
    Center_SubhaloSFR = np.zeros(Target_Halo_IDs.shape)
    Center_SubhaloSpin = np.zeros((Target_Halo_IDs.shape[0],3))
    Center_SubhaloIDs = np.zeros(Target_Halo_IDs.shape)

    for i in range(len(Target_Halo_IDs)):
        Halo_ID = Target_Halo_IDs[i]
        Subhalo_MaxMasses[i],  Galaxy_nums[i], CenterSub_Index = get_subhalo_maxM(Halo_ID, SubhaloGrNr, SubhaloMass)
        Center_SubhaloVelDisp[i] = SubhaloVelDisp[CenterSub_Index]
        Center_SubhaloSFR[i] = SubhaloSFR[CenterSub_Index]
        Center_SubhaloSpin[i] = SubhaloSpin[CenterSub_Index]
        Center_SubhaloIDs[i] = Subhalo_IDs[CenterSub_Index]

    return (Target_Halo_IDs, Subhalo_MaxMasses, Target_Halo_Rs_Crit200, Target_GroupPoses, Galaxy_nums,
            Center_SubhaloVelDisp, Center_SubhaloSFR,Center_SubhaloSpin, Subhalo_IDs)

In [8]:
def Find_CenterSubhalo(HaloIDs, SubhaloGrNr, SubhaloIDs):
    CenterSubhalo_IDs = np.zeros(HaloIDs.shape)
    CenterSubhalo_Indices = np.zeros(HaloIDs.shape)
    for i in range(len(HaloIDs)):
        HaloID = HaloIDs[i]
        CenterSubhalo_Indices[i] = np.array(np.where(SubhaloGrNr == HaloID))[0][0]
        CenterSubhalo_IDs[i] = SubhaloIDs[int(CenterSubhalo_Indices[i])]
    
    return CenterSubhalo_IDs, CenterSubhalo_Indices

In [9]:
## Generate Surface Density

In [9]:
def SurfaceDens(Halo_ID, Halo_R_Crit200, Halo_Pos, bin_number):
    """ 
    Compute the surface density map for further calculations.

    Parameters:
    - Halo_ID: ID of the target halo.
    - Halo_R_Crit200: Critical radius (R_Crit200) of the halo.
    - Halo_Pos: Position of the halo center (x, y).
    - bin_number: Number of bins along each axis for the density map.

    Returns:
    - SurfaceDens_Map: 2D numpy array representing the surface density map. 10^10 M_{sun}/(ckpc/h)**2
    """
    
    with h5py.File(f'{Cutout_Path}snap_halo_{Halo_ID}.h5', 'r') as halo_hdf:
        Gas_Coordinates = halo_hdf['PartType0/Coordinates'][:]
        Gas_Mass = halo_hdf['PartType0']['Masses'][:]
        DM_Coordinates = halo_hdf['PartType1']['Coordinates'][:]
    
    x_min = Halo_Pos[0] - Halo_R_Crit200
    x_max = Halo_Pos[0] + Halo_R_Crit200
    y_min = Halo_Pos[1] - Halo_R_Crit200
    y_max = Halo_Pos[1] + Halo_R_Crit200

    bin_length = 2*Halo_R_Crit200/bin_number

    Gas_BinMass = np.histogram2d(
    Gas_Coordinates[:, 0],   Gas_Coordinates[:, 1], bins=bin_number, range=[[x_min, x_max], [y_min, y_max]], weights=Gas_Mass)[0]

    DM_BinNum = np.histogram2d(
    DM_Coordinates[:, 0], DM_Coordinates[:, 1], bins=bin_number, range=[[x_min, x_max], [y_min, y_max]])[0]

    SurfaceDens_Map = (Gas_BinMass+ DM_BinNum*DM_Particle_Mass)/bin_length**2

    return SurfaceDens_Map

In [10]:
# Define Q function
def schirmer_filter(x):
    """
    Compute the Schirmer filter Q(x).
    
    Parameters:
    x : numpy array or float
        Dimensionless radial distance (R / Rap).

    Returns:
    Q : numpy array or float
        Schirmer filter value at x.
    """
    # Avoid division by zero using np.where
    x_safe = np.where(x == 0, 1e-10, x)  # Replace x == 0 with a very small value
    # Compute Q(x)
    Q = (1 / (1 + np.exp(6 - 150 * x) + np.exp(50 * x - 47))) * \
        (np.tanh(x_safe / 0.15) / (x_safe / 0.15))
    
    return Q

In [11]:
# Read target halo info
with h5py.File(Target_cat_name, 'a') as Target_hdf:
    FOF_Halo_IDs =  Target_hdf['Group/FOF_Halo_IDs'][:].astype(int)
    GroupPos = Target_hdf['Group/GroupPos'][:]
    Group_R_Crit200 = Target_hdf['Group/Group_R_Crit200'][:]
    Group_M_Crit200 = Target_hdf['Group/Group_M_Crit200'][:]

In [13]:
# Loop through all target halos and compute M_ap
c = const.c.value  # Speed of light in m/s
G = const.G.value  # Gravitational constant in m^3/kg/s^2

# Compute angular diameter distances
D_d = cosmo.angular_diameter_distance(Redshift).value  # Mpc
D_s = cosmo.angular_diameter_distance(1.0).value  # Mpc
D_ds = cosmo.angular_diameter_distance_z1z2(Redshift, 1.0).value  # Mpc

# Calculate critical surface density (kg/m^2)
Sigma_crit = (c**2 / (4 * np.pi * G)) * (D_s / (D_d * D_ds))
Sigma_crit *= 1e-3 /(mass_sun_in_kg*meters_to_kpc)  # Convert to M_sun/kpc^2
Sigma_crit *= M_to_e10Mperh/(kpc_to_ckpcperh)**2 # Convert to 10^(10) M_{sun}/h/(ckpc/h)**2

In [14]:
def get_kappa_map(SurfaceDens_Map,Sigma_crit):
    kappa_map = SurfaceDens_Map / Sigma_crit

    return kappa_map

In [15]:
def kappa_to_shear(kappa_map):
    """
    Convert a convergence (kappa) map to shear (gamma1, gamma2) maps using Fourier-space relations.

    Parameters:
        kappa_map: 2D numpy array

    Returns:
        gamma1, gamma2: 2D numpy arrays (same shape as kappa_map)
    """
    ny, nx = kappa_map.shape
    kx = np.fft.fftfreq(nx).reshape(1, -1)
    ky = np.fft.fftfreq(ny).reshape(-1, 1)
    kx, ky = np.meshgrid(kx, ky)

    ksq = kx**2 + ky**2
    ksq[ksq == 0] = 1  # avoid division by zero at (0,0)

    D1 = (kx**2 - ky**2) / ksq
    D2 = (2 * kx * ky) / ksq

    kappa_fft = fft2(kappa_map)

    gamma1_fft = D1 * kappa_fft
    gamma2_fft = D2 * kappa_fft
    
    # The result should be real; imaginary part is numerical noise
    gamma1 = np.real(ifft2(gamma1_fft))
    gamma2 = np.real(ifft2(gamma2_fft))

    return gamma1, gamma2


def generate_ellipticity_from_kappa_full(
    kappa_map,
    n_gal_arcmin2=20,
    sigma_e=0.3,
    pixel_size_arcmin=0.2
):
    """
    From kappa_map, simulate galaxy ellipticities e1, e2 including shear + shape noise.

    Returns:
        galaxy_positions: (N_gal, 2) positions in pixel units
        e1, e2: ellipticity components
    """
    gamma1_map, gamma2_map = kappa_to_shear(kappa_map)
    shape = kappa_map.shape

    area_arcmin2 = (pixel_size_arcmin * shape[0]) * (pixel_size_arcmin * shape[1])
    n_gal_total = int(n_gal_arcmin2 * area_arcmin2)

    # Generate galaxy positions uniformly across the map
    y = np.random.uniform(0, shape[0], size=n_gal_total)
    x = np.random.uniform(0, shape[1], size=n_gal_total)
    galaxy_positions = np.stack([x, y], axis=-1)

    # Interpolate gamma1, gamma2 at galaxy positions
    xi = np.clip(x.astype(int), 0, shape[1] - 1)
    yi = np.clip(y.astype(int), 0, shape[0] - 1)

    kappa = kappa_map[yi, xi]
    gamma1 = gamma1_map[yi, xi]
    gamma2 = gamma2_map[yi, xi]

    gamma = gamma1 + 1j * gamma2
    g = gamma / (1 - kappa)

    # Add intrinsic shape noise
    e1_int = np.random.normal(0, sigma_e, size=n_gal_total)
    e2_int = np.random.normal(0, sigma_e, size=n_gal_total)
    e_int = e1_int + 1j * e2_int

    eps = (e_int + g) / (1 + np.conj(g) * e_int)

    e1 = np.real(eps)
    e2 = np.imag(eps)

    return galaxy_positions, e1, e2


def generate_ellipticity_from_kappa(
    kappa_map,
    n_gal_arcmin2=20,
    sigma_e=0.3,
    pixel_size_arcmin=0.2
):
    """
    From kappa_map, simulate galaxy ellipticities e1, e2 including shear + shape noise.

    Returns:
        galaxy_positions: (N_gal, 2) positions in pixel units
        e1, e2: ellipticity components
    """
    gamma1_map, gamma2_map = kappa_to_shear(kappa_map)
    shape = kappa_map.shape

    area_arcmin2 = (pixel_size_arcmin * shape[0]) * (pixel_size_arcmin * shape[1])
    n_gal_total = int(n_gal_arcmin2 * area_arcmin2)

    # Generate galaxy positions uniformly across the map
    y = np.random.uniform(0, shape[0], size=n_gal_total)
    x = np.random.uniform(0, shape[1], size=n_gal_total)
    galaxy_positions = np.stack([x, y], axis=-1)

    # Interpolate gamma1, gamma2 at galaxy positions
    xi = np.clip(x.astype(int), 0, shape[1] - 1)
    yi = np.clip(y.astype(int), 0, shape[0] - 1)

    gamma1 = gamma1_map[yi, xi]
    gamma2 = gamma2_map[yi, xi]


    # Add intrinsic shape noise
    e1_int = np.random.normal(0, sigma_e, size=n_gal_total)
    e2_int = np.random.normal(0, sigma_e, size=n_gal_total)


    e1 = e1_int + gamma1
    e2 = e2_int + gamma2

    return galaxy_positions, e1, e2

def generate_ellipticity_from_kappa_nonoise(
    kappa_map,
    n_gal_arcmin2=20,
    sigma_e=0.3,
    pixel_size_arcmin=0.2
):
    """
    From kappa_map, simulate galaxy ellipticities e1, e2 including shear + shape noise.

    Returns:
        galaxy_positions: (N_gal, 2) positions in pixel units
        e1, e2: ellipticity components
    """
    gamma1_map, gamma2_map = kappa_to_shear(kappa_map)
    shape = kappa_map.shape

    area_arcmin2 = (pixel_size_arcmin * shape[0]) * (pixel_size_arcmin * shape[1])
    n_gal_total = int(n_gal_arcmin2 * area_arcmin2)

    # Generate galaxy positions uniformly across the map
    y = np.random.uniform(0, shape[0], size=n_gal_total)
    x = np.random.uniform(0, shape[1], size=n_gal_total)
    galaxy_positions = np.stack([x, y], axis=-1)

    # Interpolate gamma1, gamma2 at galaxy positions
    xi = np.clip(x.astype(int), 0, shape[1] - 1)
    yi = np.clip(y.astype(int), 0, shape[0] - 1)

    gamma1 = gamma1_map[yi, xi]
    gamma2 = gamma2_map[yi, xi]

    e1 = gamma1
    e2 = gamma2

    return galaxy_positions, e1, e2

In [16]:
def compute_SN_map(
    x0_grid,  # M x 2 array of positions (x0, y0) where you want to compute S/N
    y0_grid,
    galaxy_positions,  # N x 2 array of galaxy positions (x_i, y_i)
    e1,  # N-array of e1
    e2,  # N-array of e2
    Q_func,  # function Q(r) that gives the filter weight as a function of distance
    sigma_e=0.3,  # intrinsic ellipticity dispersion (per component),
    R_scale = 120
):
    """
    Compute S/N at a set of positions x0 using Eq. (24) from the literature.

    Parameters:
        x0_grid: M x 2 ndarray, positions to evaluate S/N
        galaxy_positions: N x 2 ndarray, galaxy coordinates
        e1, e2: N arrays of ellipticity components
        Q_func: function Q(r), the aperture filter
        sigma_e: float, intrinsic ellipticity dispersion (default 0.3)

    Returns:
        sn_values: M-array, S/N values at each x0
    """
    sn_values = []

    eps = e1 + 1j * e2  # complex ellipticity
    x_gal, y_gal = galaxy_positions[:, 0], galaxy_positions[:, 1]

    for x0, y0 in zip(x0_grid.ravel(), y0_grid.ravel()):
        dx = x_gal - x0
        dy = y_gal - y0
        r = np.sqrt(dx**2 + dy**2)

        # Mask galaxies where Q(r) ≠ 0 (or small radius cutoff)
        mask = r > 1e-5
        dx, dy, r, eps_use = dx[mask], dy[mask], r[mask], eps[mask]

        # Tangential ellipticity
        phi = dx + 1j * dy
        et = -np.real(eps_use * np.conj(phi) / phi)

        Q_vals =    Q_func(r/R_scale)

        # Signal and noise
        weight = Q_vals / r**2
        signal = np.sum(et * weight)
        noise_denom = np.sqrt(np.sum(Q_vals**2 / r**4))
        sn = np.sqrt(2) * signal / (sigma_e * noise_denom)

        sn_values.append(sn)

    return np.array(sn_values)

In [23]:
# main
for i in range(819,FOF_Halo_IDs.shape[0]):
# for i in range(1,2):
    # Extract halo properties for the current halo
    halo_id = FOF_Halo_IDs[i]
    halo_pos = GroupPos[i]
    halo_r = Group_R_Crit200[i]
    halo_m = Group_M_Crit200[i]/h_atz # unit 10^10 solar masses
    
    n_gal_arcmin2 = 20* (5*1e+4/halo_m)**2

    # calculating pixel number
    R_200 = halo_r/kpc_to_ckpcperh * u.kpc
    #bin_numer = R_200/5
    bin_number = 240

    # calulating the pixel area
    z = 0.08
    DA = cosmo.angular_diameter_distance(z)  # Mpc
    DA_kpc = DA.to(u.kpc)
    
    theta_rad = (R_200 / DA_kpc) *u.rad         # radians, no need to .to(u.rad)
    theta_arcmin = theta_rad.to(u.arcmin)
    
    # the area of every pixel
    pixel_size_arcmin = theta_arcmin / bin_number
    pixel_area_arcmin2 = pixel_size_arcmin**2
    
    # Compute the surface density map for the current halo
    SurfaceDens_Map = SurfaceDens(halo_id, halo_r, halo_pos, bin_number)
    kappa_map = get_kappa_map(SurfaceDens_Map,Sigma_crit)

    # --- Step 1: Generate galaxy ellipticities from kappa
    # galaxy_positions, e1, e2 = generate_ellipticity_from_kappa(kappa_map, pixel_size_arcmin=pixel_size_arcmin.value)
    galaxy_positions_full, e1_full, e2_full = generate_ellipticity_from_kappa_full(kappa_map, n_gal_arcmin2=n_gal_arcmin2,sigma_e=0.3,pixel_size_arcmin=pixel_size_arcmin.value)
    galaxy_positions_nonoise, e1_nonoise, e2_nonoise = generate_ellipticity_from_kappa_nonoise(kappa_map, n_gal_arcmin2=n_gal_arcmin2,sigma_e=0.3,pixel_size_arcmin=pixel_size_arcmin.value)
    
    # --- Step 2: Define halo center (e.g., center of the map)
    ny, nx = kappa_map.shape
    halo_center = np.array([nx // 2, ny // 2])
    
    # --- Step 3: Build 240 x 240 bin grid centered on halo
    grid_size = bin_number
    x_grid_vals = np.linspace(halo_center[0] - grid_size//2, halo_center[0] + grid_size//2, grid_size)
    y_grid_vals = np.linspace(halo_center[1] - grid_size//2, halo_center[1] + grid_size//2, grid_size)
    x0_grid, y0_grid = np.meshgrid(x_grid_vals, y_grid_vals)
    
    # --- Step 4: Compute S/N map
    # sn_values = compute_SN_map(x0_grid, y0_grid, galaxy_positions, e1, e2, schirmer_filter, sigma_e=0.3)
    # sn_values = sn_values.reshape(grid_size, grid_size)
    sn_values_full = compute_SN_map(x0_grid, y0_grid, galaxy_positions_full, e1_full, e2_full, schirmer_filter, sigma_e=0.3, R_scale = bin_number/2)
    sn_values_full = sn_values_full.reshape(grid_size, grid_size)
    sn_values_nonoise = compute_SN_map(x0_grid, y0_grid, galaxy_positions_nonoise, e1_nonoise, e2_nonoise, schirmer_filter, sigma_e=0.3, R_scale = bin_number/2)
    sn_values_nonoise = sn_values_nonoise.reshape(grid_size, grid_size)

    hdu_full = fits.PrimaryHDU(sn_values_full)
    hdul_full = fits.HDUList([hdu_full])
    hdul_full.writeto(f'sn_map_full_halo{halo_id}.fits', overwrite=True)

    hdu_nonoise = fits.PrimaryHDU(sn_values_nonoise)
    hdul_nonoise = fits.HDUList([hdu_nonoise])
    hdul_nonoise.writeto(f'sn_map_nonoise_halo{halo_id}.fits', overwrite=True)

    print(f'finish halo index {i} with halo id {halo_id}')

finish halo index 1 with halo id 1
